# LSSTCam pointing drift

This notebook calculates the drift of an image due to the pointing error.

Craig Lage - 29-Oct-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.obs.lsst import LsstCam
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTCam")
instrument = 'LSSTCam'
camera = LsstCam.getCamera()
client = makeEfdClient()

In [ ]:
def DeltaAltAz(ra, dec, pressure, hum, temperature, wl, time1, time2, printOut=False):
    # This calculates the change in AltAz during an exposure
    # given the RA/Dec and other variables
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # el, az are the actual values, prime values reflect the pointing model
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    if printOut:
        print(f"AzStart={az1:.6f}, ElStart={el1:.6f}, AzEnd={az2:.6f}, ElEnd={el2:.6f}")
        print(f"AzChange={azChange:.2f}, elChange={elChange:.2f}")
    return [azChange, elChange]

In [ ]:
# Below is the wavelength center point for the LSST filters:
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}
detector = camera['R22_S11']
bbox = detector.getBBox()

def CalculateDrift(expId, printOut=False):
    # Get the azChange and elChange from the Headers
    rawExp = butler.get('raw', detector=94, exposure=expId, instrument=instrument)
    md = rawExp.getMetadata()
    azChangePoint = (md['AZEND'] - md['AZSTART']) * 3600.0
    elChangePoint = (md['ELEND'] - md['ELSTART']) * 3600.0
    filter = md['FILTBAND']
    wl = wavelengths[filter] * u.angstrom
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY'] / 100.0
    time1 = Time(md['MJD-BEG'], format='mjd', scale='tai')
    time2 = Time(md['MJD-END'], format='mjd', scale='tai')
    raPoint = md['RA']
    decPoint = md['DEC']
    el = md['ELSTART']
    calExp = butler.get('preliminary_visit_image', detector=94, visit=expId, instrument=instrument)
    cWcs = calExp.getWcs()
    if not cWcs:
        return None
    calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    raReal = calExpSkyCenter.getRa().asDegrees()
    decReal = calExpSkyCenter.getDec().asDegrees()
    deltaRa = (raReal - raPoint) * 3600.0
    deltaDec = (decReal - decPoint) * 3600.0
    [azChangeReal, elChangeReal] = DeltaAltAz (raReal, decReal, pressure, hum, temperature, wl, time1, time2, printOut)
    azDrift = azChangeReal - azChangePoint
    azDrift *= np.cos(el * np.pi / 180.0)
    elDrift = elChangeReal - elChangePoint
    totalDrift = np.sqrt(elDrift**2 + azDrift**2)
    if printOut:
        print(f"We think telescope is pointed at (RA, Dec), ({raPoint:.6f}, {decPoint:.6f})")
        print(f"Telescope is actually pointed at (RA, Dec), ({raReal:.6f}, {decReal:.6f})")
        print(f"Pointing error in RA, Dec is ({deltaRa:.1f}, {deltaDec:.1f}) arcseconds")
        print(f"azChangeHeader = {azChangePoint:.2f}, elChangeHeader = {elChangePoint:.2f}")
        print(f"azChangeReal = {azChangeReal:.2f}, elChangeReal = {elChangeReal:.2f}")
        print(f"For {expId}, Azimuth drift = {azDrift:.2f} arcseconds, Elevation drift = {elDrift:.2f} arcseconds, Total drift = {totalDrift:.2f} arcseconds.")
        print(f"Header AzStart={md['AZSTART']:.6f}, AzEnd={md['AZEND']:.6f}, ElStart={md['ELSTART']:.6f}, ElEnd={md['ELEND']:.6f}")
        print(f"Header AzChange={((md['AZEND'] - md['AZSTART'])*3600):.6f}, ElChange={((md['ELEND'] - md['ELSTART'])*3600):.6f}")
    return [azDrift, elDrift]

## Get the guider drift data from RubinTV and calculate the ptg drift

In [ ]:
startDay = 20251026
endDay = 20251027
azs = []
els = []
az_drifts = []
el_drifts = []
total_drifts = []
ptg_az_drifts = []
ptg_el_drifts = []
ptg_total_drifts = []
expIds = []
dayObs = startDay
while dayObs <= endDay:
        try:
            guiderTable = pd.read_json(f'/home/c/cslage/u/LSSTCam/rubintv_data/dayObs_{dayObs}.json').T
            guiderTable = guiderTable.sort_index()
            print(dayObs, len(guiderTable))
            for i in range(1, len(guiderTable)+1):
                if i<100:
                    continue
                expId = int(dayObs * 1.0E5 + i)
                try:
                    expTime = float(guiderTable.loc[i]['Exposure time'])
                    az = float(guiderTable.loc[i]['Azimuth'])
                    el = float(guiderTable.loc[i]['Elevation'])
                    rot = float(guiderTable.loc[i]['Sky angle'])
                    az_drift = float(guiderTable.loc[i]['Az drift (arcsec total)'])
                    el_drift = float(guiderTable.loc[i]['Alt drift (arcsec total)'])
                    rot_drift = float(guiderTable.loc[i]['Rotator drift (arcsec total)'])
                    az_drift *= expTime
                    el_drift *= expTime
                    rot_drift *= expTime
                    data = np.array([az, el, rot, az_drift, el_drift, rot_drift])
                    if np.isnan(data).any():
                        print(f"{expId} had nans!")
                        continue
                    [ptg_az_drift, ptg_el_drift] = CalculateDrift(expId)
                    if not ptg_az_drift:
                        print(f"{expId} had no WCS!")
                        continue
                    expIds.append(expId)
                    azs.append(az)
                    els.append(el)
                    az_drifts.append(az_drift)
                    el_drifts.append(el_drift)
                    total_drift = np.sqrt(el_drift**2 + az_drift**2)
                    total_drifts.append(total_drift)
                    ptg_az_drifts.append(ptg_az_drift)
                    ptg_el_drifts.append(ptg_el_drift)
                    ptg_total_drift = np.sqrt(ptg_el_drift**2 + ptg_az_drift**2)
                    ptg_total_drifts.append(ptg_total_drift)
                    print(f"{expId} succeeded!")
                except:
                    print(f"{expId} failed!")
                    continue
        except:
            print(f"{dayObs} failed")
            dayObs = calcNextDay(dayObs)
            continue
        print(f" Done with {dayObs}")
        dayObs = calcNextDay(dayObs)     
print(len(expIds), len(azs), len(els), len(az_drifts), len(el_drifts),
      len(total_drifts), len(ptg_az_drifts), len(ptg_el_drifts),
      len(ptg_total_drifts))
filename = f"/home/c/cslage/u/LSSTCam/guider_data/guider_drifts_header_{startDay}_{endDay}.pkl"
with open(filename, 'wb') as f:
    pkl.dump([expIds, azs, els, az_drifts, el_drifts, total_drifts, ptg_az_drifts, ptg_el_drifts, ptg_total_drifts], f)


In [ ]:
plt.figure(figsize=(12,5))
plt.suptitle(f"Guider drifts vs Pointing Error drifts {startDay}-{endDay}")
plt.subplot(1,2,1)
plt.title("Azimuth drift/exposure")
plt.plot(np.array(ptg_az_drifts), marker='x', ms=0.1, label="Ptg error")
plt.plot(az_drifts, marker='o', ms=0.1, alpha=0.5, label="Guider")
plt.xlabel("Exposure list index")
plt.ylabel("Drift/exposure (arcsec)")
plt.legend(loc = 'lower left')
plt.subplot(1,2,2)
plt.title("Elevation drift/exposure")
plt.plot(np.array(ptg_el_drifts), marker='x', ms=0.1)
plt.plot(el_drifts, marker='o', ms=0.1, alpha=0.5)
plt.xlabel("Exposure list index")
plt.ylabel("Drift/exposure (arcsec)")
plt.savefig(f"/home/c/cslage/u/LSSTCam/guider_data/Guider_Drifts_Pointing_Header_{startDay}_{endDay}.png")

## Plot the total drifts

In [ ]:
plt.figure(figsize=(10,5))
plt.suptitle(f"Guider drifts vs Pointing Error drifts {startDay}-{endDay}")
plt.title("Total drift/exposure")
plt.plot(np.array(ptg_total_drifts)*0.8, marker='x', ms=0.1, label="Ptg error")
plt.plot(total_drifts, marker='o', ms=0.1, alpha=0.5, label="Guider")
plt.xlabel("Exposure list index")
plt.ylabel("Drift/exposure (arcsec)")
plt.legend(loc = 'lower left')
plt.savefig(f"/home/c/cslage/u/LSSTCam/guider_data/Guider_Drifts_Total_Header_{startDay}_{endDay}.png")

In [ ]:
i = 155
print(expIds[i], az_drifts[i], el_drifts[i], total_drifts[i], ptg_az_drifts[i],
      ptg_el_drifts[i], ptg_total_drifts[i])

In [ ]:
i = 156
print(expIds[i], az_drifts[i], el_drifts[i], total_drifts[i], ptg_az_drifts[i],
      ptg_el_drifts[i], ptg_total_drifts[i])

In [ ]:
i = 220
print(expIds[i], az_drifts[i], el_drifts[i], total_drifts[i], ptg_az_drifts[i],
      ptg_el_drifts[i], ptg_total_drifts[i])

In [ ]:
i = 300
print(expIds[i], az_drifts[i], el_drifts[i], total_drifts[i], ptg_az_drifts[i],
      ptg_el_drifts[i], ptg_total_drifts[i])

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.scatter(az_drifts, ptg_az_drifts)
fit = np.polyfit(az_drifts, ptg_az_drifts, 1)
print(fit)
xs = np.linspace(-0.5, 0.5, 100)
ys = np.polyval(fit, xs)
plt.plot(xs, ys, ls='--', color='red')
plt.subplot(1,2,2)
plt.scatter(el_drifts, ptg_el_drifts)
fit = np.polyfit(el_drifts, ptg_el_drifts, 1)
print(fit)
xs = np.linspace(-0.5, 0.5, 100)
ys = np.polyval(fit, xs)
plt.plot(xs, ys, ls='--', color='red')

In [ ]:
[azDrift, elDrift] = CalculateDrift(2025102600225, printOut=True)

In [ ]:
expId = 2025102600155
[azDrift, elDrift] = CalculateDrift(expId, printOut=True)

In [ ]:
expId = 2025102600156
[azDrift, elDrift] = CalculateDrift(expId, printOut=True)

In [ ]:
expId = 2025102600225
[azDrift, elDrift] = CalculateDrift(expId, printOut=True)

In [ ]:
expId = 2025102600226
[azDrift, elDrift] = CalculateDrift(expId, printOut=True)